# Intro to tensorflow

In [1379]:
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

In [1380]:
a = tf.constant(5)
b = tf.constant(7)
add = tf.add(a, b)
sess = tf.Session()
output = sess.run(add)
print(output) # 12
sess.close()

12


In [1381]:
c = tf.placeholder(tf.int32)
d = tf.placeholder(tf.int32)
add = tf.add(c, d)
sess = tf.Session()
output = sess.run(add, feed_dict={c:5, d:7})
print(output) # 12
output = sess.run(add, feed_dict={c:20, d:32})
print(output) # 52
sess.close()


12
52


### NOTE:
Following tensorflow 2 from now on.

# Logistic regress on Tensorflow

In [1382]:
import numpy as np
x_train = np.array([[0,0],[0,1],[1,0],[1,1]])
y_train = np.array([[0],[0],[0],[1]])

In [1383]:
x = tf.placeholder(tf.float32, [None, 2])
t = tf.placeholder(tf.float32, [None, 1])

In [1384]:
W = tf.Variable(tf.zeros([2,1]))
b = tf.Variable(tf.zeros([1]))

In [1385]:
y = tf.sigmoid(tf.matmul(x, W) + b)
cross_entropy = tf.reduce_sum(-t * tf.log(y) - (1 - t) * tf.log(1 - y))

In [1386]:
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy)

In [1387]:
correct_prediction = tf.equal(tf.sign(y - 0.5), tf.sign(t - 0.5))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [1388]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [1389]:
for epoch in range(1000):
    sess.run(train_step, feed_dict={
        x:x_train,
        t:y_train
    })
# Display the correct answer rate every 100 times
    if epoch % 100 == 0:
        acc_val = sess.run(
            accuracy, feed_dict={
                x:x_train,
                t:y_train})
        print ('epoch: %d, Accuracy: %f'
               %(epoch, acc_val))

epoch: 0, Accuracy: 0.750000
epoch: 100, Accuracy: 1.000000
epoch: 200, Accuracy: 1.000000
epoch: 300, Accuracy: 1.000000
epoch: 400, Accuracy: 1.000000
epoch: 500, Accuracy: 1.000000
epoch: 600, Accuracy: 1.000000
epoch: 700, Accuracy: 1.000000
epoch: 800, Accuracy: 1.000000
epoch: 900, Accuracy: 1.000000


In [1390]:
#Check if the learning result is correct
classified = sess.run(correct_prediction, feed_dict={
    x:x_train,
    t:y_train
})
#Check output y
prob = sess.run(y, feed_dict={
    x:x_train,
    t:y_train
})
print(classified)
# [[ True]
# [ True]
# [ True]
# [ True]]

print(prob)
# [[  1.96514215e-04]
# [  4.90498319e-02]
# [  4.90498319e-02]
# [  9.31203783e-01]]

[[ True]
 [ True]
 [ True]
 [ True]]
[[1.9654632e-04]
 [4.9049824e-02]
 [4.9049824e-02]
 [9.3120384e-01]]


In [1391]:
print('W:', sess.run(W))
print('b:', sess.run(b))
# W: [[ 5.5699544]
# [ 5.5699544]]
# b: [-8.53457928]

W: [[5.5699544]
 [5.5699544]]
b: [-8.534579]


In [1392]:
sess.close()

# Tensorflow (v1)

# Problem 1
Looking back on the scratch

### Steps to build a ML model
1. Load
2. Define model (class) along with needed stuffs (params, input shapes,..., procedure, batch ...)
3. Init stuff and use the model

# Problem 2
Correspond between scratch and tensorflow

In [1393]:
"""
Binary classification of Iris dataset using neural network implemented in TensorFlow
"""
import pandas as pd
from sklearn.model_selection import train_test_split

#Load dataset
df = pd.read_csv("Iris.csv")

#Condition extraction from data frame
df = df[(df["Species"] == "Iris-versicolor") | (df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
# NumPy 配列に変換
X = np.array(X)
y = np.array(y)
# Convert label to number
y[y == "Iris-versicolor"] = 0
y[y == "Iris-virginica"] = 1
y = y.astype(np.int64)[:, np.newaxis]
#Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
class GetMiniBatch:
    """
    Iterator to get a mini-batch
    Parameters
    ----------
    X : The following forms of ndarray, shape (n_samples, n_features)
      Training data
    y : The following form of ndarray, shape (n_samples, 1)
      Correct answer value
    batch_size : int
      Batch size
    seed : int
      NumPy random number seed
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]
# Hyperparameter settings
learning_rate = 0.001
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1
#Determine the shape of the argument to be passed to the calculation graph
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# train mini batch iterator
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
def example_net(x):
    """
    Simple 3-layer neural network
    """
    tf.random.set_random_seed(0)
    # Declaration of weight and bias
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.add and + are equivalent
    return layer_output
#Read network structure                              
logits = example_net(X)
# Objective function
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
# Optimization method
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# Estimated result
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
#Indicator value calculation
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
#Initialization of variable
init = tf.global_variables_initializer()

#Run calculation graph
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        #Loop for each epoch
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int64)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # Loop for each mini-batch
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})            
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
        total_loss /= n_samples

        val_loss, acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}".format(epoch, total_loss, val_loss, acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))


Epoch 0, loss : 8.7645, val_loss : 94.4552, acc : 0.375
Epoch 1, loss : 6.1416, val_loss : 62.6028, acc : 0.375
Epoch 2, loss : 3.5133, val_loss : 30.6852, acc : 0.375
Epoch 3, loss : 1.1685, val_loss : 2.5341, acc : 0.625
Epoch 4, loss : 0.9549, val_loss : 3.6983, acc : 0.688
Epoch 5, loss : 0.3575, val_loss : 5.3783, acc : 0.375
Epoch 6, loss : 0.4797, val_loss : 4.2005, acc : 0.375
Epoch 7, loss : 0.3030, val_loss : 0.5185, acc : 0.812
Epoch 8, loss : 0.2119, val_loss : 0.7033, acc : 0.812
Epoch 9, loss : 0.1795, val_loss : 1.6276, acc : 0.625
test_acc : 0.750


### Steps to build in tensorflow
1. Load dataset
2. Define model and it's flow
   1. Define parameters (variable) and inputs (placeholders)
   2. Define model's prediction (or forward flow)
   3. Define models' loss function
   4. Get an optimizer
   5. Define the traing flow
      - Optimize
      - Training scores
      - Loss
      - Log,...
3. Prepare epoch and batch
4. Initialize paremeters
5.  Start a session
  - Train in batches
  - Use the model and stuss defined in tf syntax
6.  Close session

**Tensorflow 1.0 has a lazy execution approach to it.**

# Problem 3
Create a model of Iris using all three types of objective variables

## NOTE
**Let's go through the process one by one and test them all!**

## Data Prep

In [1394]:
#load dataset
df = pd.read_csv('Iris.csv')

print('shape: ', df.shape)
print(df.head(5))

shape:  (150, 6)
   Id  SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm      Species
0   1            5.1           3.5            1.4           0.2  Iris-setosa
1   2            4.9           3.0            1.4           0.2  Iris-setosa
2   3            4.7           3.2            1.3           0.2  Iris-setosa
3   4            4.6           3.1            1.5           0.2  Iris-setosa
4   5            5.0           3.6            1.4           0.2  Iris-setosa


In [1395]:
# predictor
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
print('X shape:', X.shape)
print(X.head(5))

X shape: (150, 4)
   SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm
0            5.1           3.5            1.4           0.2
1            4.9           3.0            1.4           0.2
2            4.7           3.2            1.3           0.2
3            4.6           3.1            1.5           0.2
4            5.0           3.6            1.4           0.2


In [1396]:
# label
y = df['Species']
print('y shape: ', y.shape)
print(y.head(5))

y shape:  (150,)
0    Iris-setosa
1    Iris-setosa
2    Iris-setosa
3    Iris-setosa
4    Iris-setosa
Name: Species, dtype: object


In [1397]:
# Convert to numpy
X = np.array(X)
y = np.array(y)
print(X.shape, y.shape)

(150, 4) (150,)


In [1398]:
# Encode y from text to number label
print('init shape y: ', y.shape)
#label encode
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
y = LabelEncoder().fit_transform(y)
#reshape for onehot use
y = y.reshape(-1,1)
enc = OneHotEncoder()
y = enc.fit_transform(y).toarray()
print('shape: ', y.shape)
print(y[:5])

init shape y:  (150,)
shape:  (150, 3)
[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]


In [1399]:
from sklearn.model_selection import train_test_split
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.9)
print('Train/Test size: ', [data.shape for data in [X_train, X_test, y_train, y_test]])
# train validation split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, train_size = 0.8)
print('Train/Test size: ', [data.shape for data in [X_train, X_val, y_train, y_val]])

Train/Test size:  [(135, 4), (15, 4), (135, 3), (15, 3)]
Train/Test size:  [(108, 4), (27, 4), (108, 3), (27, 3)]


## Model Prep

In [1400]:
# some variables
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = y_train.shape[1]

print('input nodes: ', n_input)
print('sample size: ', n_samples)
print('prediction classes: ', n_classes)

input nodes:  4
sample size:  108
prediction classes:  3


In [1401]:
# train mini batch iterator
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

print('minibatch prepared!')

minibatch prepared!


In [1402]:
# Hyperparameter settings
learning_rate = 0.001
batch_size = 10
num_epochs = 100
n_hidden1 = 50
n_hidden2 = 100

print('HYPER PARAMS')
print('alpha learning rate: ', learning_rate)
print('batch size: ', batch_size)
print('epoch num', num_epochs)
print('n_hidden1: ', n_hidden1)
print('n_hidden2: ', n_hidden2)

HYPER PARAMS
alpha learning rate:  0.001
batch size:  10
epoch num 100
n_hidden1:  50
n_hidden2:  100


In [1403]:
#Determine the shape of the argument to be passed to the calculation graph
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
print('X holder: ',X)
print('Y holder: ', Y)

X holder:  Tensor("Placeholder_330:0", shape=(None, 4), dtype=float32)
Y holder:  Tensor("Placeholder_331:0", shape=(None, 3), dtype=float32)


# Model

### Test layer

In [1404]:
# weight
w1 = tf.Variable(tf.random_normal([n_input, n_classes]))
b1 = tf.Variable(tf.random_normal([n_classes]))
print('w:', w1)
print('b:', b1)

w: <tf.Variable 'Variable_501:0' shape=(4, 3) dtype=float32>
b: <tf.Variable 'Variable_502:0' shape=(3,) dtype=float32>


In [1405]:
# layer
print('X: ', X)
layer1 = tf.matmul(X, w1)
print('after matrix multiplication:', layer1)
layer1 = tf.add(layer1, b1)
print('after adding bias:', layer1)
layer1 = tf.nn.relu(layer1)
print('after activation: ', layer1)

X:  Tensor("Placeholder_330:0", shape=(None, 4), dtype=float32)
after matrix multiplication: Tensor("MatMul_255:0", shape=(None, 3), dtype=float32)
after adding bias: Tensor("Add_357:0", shape=(None, 3), dtype=float32)
after activation:  Tensor("Relu_142:0", shape=(None, 3), dtype=float32)


In [1406]:
session = tf.Session()
def do(stuff, input):
  return session.run(stuff, feed_dict = input)
def prep_session():
  session.run(tf.global_variables_initializer())

prep_session()
forward = do(layer1, {X: X_train, Y: y_train})
print('forward shape: ', forward.shape)
print('forward: ', forward[0])


forward shape:  (108, 3)
forward:  [ 0.        13.25295    1.8706096]


## Objective Function

In [1407]:
logits = layer1 # assuming 1 layer model
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))
print('loss:', loss_op)


# the equivalent or the tf.softmax's implementation!
softmax = tf.nn.softmax(logits)
loss_op2 = -tf.reduce_sum(Y * tf.log(softmax), 1)
loss_op2 = tf.reduce_mean(loss_op2)

loss: Tensor("Mean_205:0", shape=(), dtype=float32)


In [1408]:
# test loss op
tf.set_random_seed(0)

x_in, y_in = X_train[:5], y_train[:5]
run = lambda x: do(x, {X: x_in, Y: y_in})
print('y_true: ', y_in)

loss = run(loss_op)
print('loss: ', loss)
loss2 = run(loss_op2)
print('loss2: ', loss2.mean())

y_true:  [[0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]]
loss:  4.027909
loss2:  4.027909


## Prediction

In [1409]:
def prediction_function(logits):
  pred = np.argmax(logits, axis = 1).reshape(-1,1)
  return enc.transform(pred).toarray()
logits_ = run(logits)
print('logits: ', logits_)
print('pred: ', prediction_function(logits_))


logits:  [[ 0.        13.25295    1.8706096]
 [ 0.        13.412225   1.9192337]
 [ 0.        12.519266   1.6441416]
 [ 0.        13.583306   1.5877434]
 [ 0.         8.7562685  1.5236439]]
pred:  [[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]


## Optimizer

In [1410]:
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
print(optimizer)

## Some Metrics

In [1411]:
# accuracy
def accuracy_function(pred, real):
  correct_pred = np.argmax(pred,axis = 1) == np.argmax(real,axis = 1)
  return correct_pred.mean()
pred = prediction(logits_)
real = y_in
print('accuracy_function: ', accuracy_function(pred,real))

accuracy_function:  0.6


## Training

In [1412]:
train_optimizer = optimizer.minimize(loss_op)

def accuracy(logits, y):
  prediction = prediction_function(logits)
  acc = accuracy_function(prediction, y)
  return acc

def train_batch(mini_batch_x, mini_batch_y):
  input_xy = {X: mini_batch_x, Y: mini_batch_y}
  run = lambda x: do(x, input_xy)
  # train
  run(train_optimizer)
  # loss
  batch_loss= run(loss_op)
  # accuracy
  batch_acc = accuracy(run(logits), mini_batch_y)
  return batch_loss, batch_acc

def train_epoch(X_val, y_val, iterator, verbose= False):
  #Loop for each epoch
  total_loss = 0
  total_acc = 0
  # train
  for i, (mini_batch_x, mini_batch_y) in enumerate(iterator):
      batch_loss, batch_acc = train_batch(mini_batch_x, mini_batch_y)
      total_loss += batch_loss
      total_acc += batch_acc * batch_size
      if verbose:
        print("Batch: {}, b_loss: {:.3f}, b_acc: {:.3f}".format(i, batch_loss, batch_acc))
  total_loss /= n_samples
  total_acc /= n_samples

  input_xy = {X: X_val, Y: y_val}
  run = lambda x: do(x, input_xy)
  val_loss = run(loss_op)
  val_acc = accuracy(run(logits), y_val)
  return total_loss, total_acc, val_loss, val_acc

def train(X_train, y_train, X_val, y_val, epoch = 10, verbose_interval = 10):
  iterator = GetMiniBatch(X_train, y_train, batch_size=batch_size)
  for i in range(epoch):
    # print('Starting epoch: ', i)
    total_loss, total_acc, val_loss, val_acc = train_epoch(X_val, y_val, iterator)
    if (i+1)%verbose_interval == 0 or i == 0:
      print("Epoch {} Result -- tot_loss : {:.2f}, tot_acc : {:.2f}, val_loss : {:.2f}, val_acc : {:.2f}".format(i, total_loss, total_acc, val_loss, val_acc))

    # print('______')

session.close()
session = tf.Session()
prep_session()
train(X_train, y_train, X_val, y_val, epoch = 100, verbose_interval = 10)

Epoch 0 Result -- tot_loss : 0.74, tot_acc : 0.34, val_loss : 7.05, val_acc : 0.33
Epoch 9 Result -- tot_loss : 0.58, tot_acc : 0.34, val_loss : 5.58, val_acc : 0.33
Epoch 19 Result -- tot_loss : 0.40, tot_acc : 0.34, val_loss : 3.99, val_acc : 0.33
Epoch 29 Result -- tot_loss : 0.26, tot_acc : 0.34, val_loss : 2.80, val_acc : 0.33
Epoch 39 Result -- tot_loss : 0.22, tot_acc : 0.38, val_loss : 2.45, val_acc : 0.48
Epoch 49 Result -- tot_loss : 0.21, tot_acc : 0.47, val_loss : 2.27, val_acc : 0.44
Epoch 59 Result -- tot_loss : 0.19, tot_acc : 0.46, val_loss : 2.08, val_acc : 0.44
Epoch 69 Result -- tot_loss : 0.17, tot_acc : 0.47, val_loss : 1.88, val_acc : 0.44
Epoch 79 Result -- tot_loss : 0.16, tot_acc : 0.47, val_loss : 1.68, val_acc : 0.37
Epoch 89 Result -- tot_loss : 0.14, tot_acc : 0.48, val_loss : 1.48, val_acc : 0.37
Epoch 99 Result -- tot_loss : 0.12, tot_acc : 0.49, val_loss : 1.28, val_acc : 0.37


# Problem 4
Creating a model of house prices

In [1413]:
# data set
house_price_pd = pd.read_csv('../Data/Normal/houseprice_train.csv')
house_price_pd.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [1414]:
# selection
selected = house_price_pd[['GrLivArea', 'YearBuilt','SalePrice']]
print('selected: ', selected.shape, selected.columns)
print(selected.head())

selected:  (1460, 3) Index(['GrLivArea', 'YearBuilt', 'SalePrice'], dtype='object')
   GrLivArea  YearBuilt  SalePrice
0       1710       2003     208500
1       1262       1976     181500
2       1786       2001     223500
3       1717       1915     140000
4       2198       2000     250000


In [1415]:
# check nan
print(selected.isna().sum(axis = 0))

GrLivArea    0
YearBuilt    0
SalePrice    0
dtype: int64


In [1416]:
# normalize
print(selected.describe().transpose()[['mean', 'std']])

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled = scaler.fit_transform(selected)
print(pd.DataFrame(scaled).describe().transpose()[['mean', 'std']])

explainatory = scaled[:,:-1]
objective = scaled[:,-1].reshape(-1,1)
print(explainatory.shape, explainatory[:3])
print(objective.shape, objective[:3])

                    mean           std
GrLivArea    1515.463699    525.480383
YearBuilt    1971.267808     30.202904
SalePrice  180921.195890  79442.502883
           mean       std
0 -1.277517e-16  1.000343
1  1.046347e-15  1.000343
2  1.362685e-16  1.000343
(1460, 2) [[ 0.37033344  1.05099379]
 [-0.48251191  0.15673371]
 [ 0.51501256  0.9847523 ]]
(1460, 1) [[0.34727322]
 [0.00728832]
 [0.53615372]]


In [1417]:
# train test split
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(explainatory, objective, train_size = 0.8)
print('train shape: ', [i.shape for i in [X_train, X_val, y_train, y_val]])

train shape:  [(1168, 2), (292, 2), (1168, 1), (292, 1)]


In [1418]:
# prep vars
input_dimention = X_train.shape[-1]
batch_size = 20
lr = 0.1


In [1419]:
# placeholder
X = tf.placeholder("float", [None, input_dimention])
Y = tf.placeholder("float", [None, 1])
print(X,Y)

Tensor("Placeholder_332:0", shape=(None, 2), dtype=float32) Tensor("Placeholder_333:0", shape=(None, 1), dtype=float32)


In [1427]:
# model
w = tf.Variable(tf.random_normal([input_dimention, 1]))
b = tf.Variable(tf.random_normal([1]))
linear = tf.matmul(X, w)
linear = tf.add(linear, b)

model = linear

In [1428]:
# loss
loss = Y - model
loss = tf.math.square(loss)
loss = tf.math.reduce_mean(loss) / 2

# optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr)
optimize_operation = optimizer.minimize(loss)

In [1434]:
# train
def _train(X_train, y_train, X_val, y_val):
  input_xy = {X: X_train, Y: y_train}
  run = lambda x: do(x, input_xy)
  # train
  run(optimize_operation)
  # predict
  train_loss =  run(loss)
  input_xy = {X: X_val, Y: y_val}
  val_loss = run(loss)
  return train_loss, val_loss

def train(X_train, y_train, X_val, y_val, epoch = 100, print_interval = 10):
  print(X_train.shape)
  for i in range(epoch): 
    train_loss, val_loss = _train(X_train, y_train, X_val, y_val)
    if (i + 1) % print_interval == 0 or i == 0:
      print(f'Ilteration: {i}, SSE_train: {train_loss}, SSE_val: {val_loss}')

session.close()
session = tf.Session()
prep_session()
train(X_train, y_train, X_val, y_val)

(1168, 2)
Ilteration: 0, SSE_train: 0.5249675512313843, SSE_val: 0.5466243028640747
Ilteration: 9, SSE_train: 0.2246502935886383, SSE_val: 0.24539430439472198
Ilteration: 19, SSE_train: 0.1761990338563919, SSE_val: 0.19716167449951172
Ilteration: 29, SSE_train: 0.16992352902889252, SSE_val: 0.19106000661849976
Ilteration: 39, SSE_train: 0.16908639669418335, SSE_val: 0.1902848184108734
Ilteration: 49, SSE_train: 0.16897033154964447, SSE_val: 0.1901824176311493
Ilteration: 59, SSE_train: 0.16895349323749542, SSE_val: 0.19016504287719727
Ilteration: 69, SSE_train: 0.1689509004354477, SSE_val: 0.19015952944755554
Ilteration: 79, SSE_train: 0.16895049810409546, SSE_val: 0.19015681743621826
Ilteration: 89, SSE_train: 0.16895043849945068, SSE_val: 0.19015537202358246
Ilteration: 99, SSE_train: 0.1689504086971283, SSE_val: 0.19015459716320038


# Problem 5
Creating a MNIST model

# TODO!